In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def f(x):
  return 3*x**2 - 4*x +5

In [ ]:
xs = np.arange(-5,5,0.25)
ys = f(xs)
plt.plot(xs,ys)

In [ ]:
# @title
# function -> 3*x**2 - 4*x +5
# derivative -> 6*x -4

h = 0.001
x = 2/3

# the defination of a derivative==>
# as we increase the input to a function by h, where h->0,
# how does the function change i.e what is it's slope
(f(x+h) - f(x))/h

0.0029999999995311555

In [ ]:
# @title
h = 0.00001

a = 2.0
b = -3.0
c = 10.0

d1 = a*b +c
c += h
d2 = a*b + c
print('d1', d1)
print('d2', d2)
print('slope', (d2-d1)/h)

In [ ]:
# @title MicroGrad
class Value:
  # _children will keep track of the variables that participates in the operation,
  # _op will keep track of the type of operation performed

  def __init__(self, data, _children=(), _op='', label=''): # _children ->empty tuple
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)  # self._prev -> empty set
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  # if don't use __rapr__ then python will print out some abstract object instead of our value

  def __add__(self,other):
    # so that we can also pass in int, float as other
    # without it, any int, float will give error since other.data can only take 'Value' object
    other = other if isinstance(other, Value) else Value(other)
    # so if input other is 'Value' object then leave it alone, else wrap the input other into a 'Value' object

    out = Value(self.data + other.data, (self, other), '+')
    # def __init__(self,     data,      _children=(), _op='')

    def _backward():
      self.grad +=  out.grad
      other.grad +=  out.grad

    out._backward = _backward
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad

    out._backward = _backward
    return out

  def __rmul__(self, other):
    # if __mul__ cannot do 2*a, it will check if there is __rmul__ defined
    # __rmul__ will swap the operand (2*a -> a*2) and then pass it back to __mul__
    return self * other

  def __radd__(self, other): # other + self
    return self + other

  def __truediv__(self, other):
    return self * (other ** -1)

  def __neg__(self):
    return self * -1

  def __sub__(self, other):
    return self + (-other)

  def __pow__(self, other):
    assert isinstance(other, (int, float)) # only supporting int/float for now
    out = Value(self.data ** other, (self, ), f'**{other}')

    def _backward():
      self.grad += (other * (self.data**(other-1))) * out.grad
      # other.grad += ((self.grad/out.data) - (other.data/self.data))* (1/math.ln(self.data)) * out.grad

    out._backward = _backward
    return out

  # def __truediv__(self, other):
  #   other = other if isinstance(other, Value) else Value(other)
  #   out = Value(self.data / other.data, (self, other), '/')
  #   def _backward():
  #     self.grad += (1/other.data) * out.grad
  #     other.grad += (-1/(other.data * other.data)) * out.grad

  #   out._backward = _backward
  #   return out

  # def __sub__(self, other):
  #   other = other if isinstance(other, Value) else Value(other)
  #   out = Value(self.data - other.data, (self, other), '-')
  #   def _backward():
  #     self.grad += 1.0 * out.grad
  #     other.grad += -1.0 * out.grad

  #   out._backward = _backward
  #   return out

  def exp(self):
    x = self.data
    t = math.exp(x)
    out = Value(t, (self, ), 'exp')
    def _backward():
      self.grad += out.data * out.grad

    out._backward = _backward
    return out

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')

    def _backward():
      self.grad += (1- (out.data)**2) * out.grad

    out._backward = _backward
    return out

  def backward(self):
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)

    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()


In [ ]:
# @title Graphing
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()

  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)

  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right

  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for anyvalue in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label= "{%s | data %.4f | grad %.4f}" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    #connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

In [ ]:
# @title simple example
a = Value(2.0, label='a')
b = Value(-3.0, label='b')
c = Value(10.0, label='c')
e = a*b ; e.label = 'e'
d = e+c ; d.label = 'd'
f = Value(-2.0, label = 'f')
L = d/f


#a + b # by defining __add__ function, we can add the two Value objects a and b togather just by using the '+' sign
      # internally python looks at the function wrapped in '__ __' and uses it as the operator
      # so now a + b is the same as a.__add__(b)
#d = a*b + c # same as --> (a.__mul__(b)).__add__(c)


# print(f"result= {L}, children= {L._prev}, operation= {L._op}")

L.grad = 1.0
d.grad = 1.0/f.data
f.grad = -d.data / (f.data*f.data)
e.grad = d.grad * 1.0 # dL/de = dL/dd * dd/de chain rule      #1/f.data
c.grad = d.grad * 1.0
a.grad = e.grad * b.data # dL/da = dL/dd * dd/de * de/da   #b.data/f.data
b.grad = e.grad * a.data     #a.data/f.data

In [ ]:
# @title optimization/small steps
# optimization step, setting a small learning rate 0.01 and going towards (+) the gradient
a.data += 0.01 * a.grad
b.data += 0.01 * b.grad
c.data += 0.01 * c.grad
f.data += 0.01 * f.grad

e = a * b
d = e + c
L = d / f
print(L.data)
# and from the result we see the gradient did indeed go up or go positive

-1.955149253731343


In [ ]:
# @title manual derivatives
def lol(): # manual derivatives
  h= 0.0001

  a = Value(2.0, label='a')
  b = Value(-3.0, label='b')
  c = Value(10.0, label='c')
  e = a*b ; e.label = 'e'
  d = e+c ; d.label = 'd'
  f = Value(-2.0, label = 'f')
  L = d/f
  L1 = L.data

  a = Value(2.0 , label='a') # nudging a by a small amount h, a+h

  b = Value(-3.0, label='b')
  c = Value(10.0, label='c')
  e = a*b ; e.label = 'e'
  d = e+c ; d.label = 'd'
  d.data += h
  f = Value(-2.0, label = 'f')

  L = d/f
  L2 = L.data

  print((L2-L1)/h,L2, L1) # dL/da, how a small change in 'a' changes the derivative of L

lol()

-0.4999999999988347 -2.00005 -2.0


# Example using a neuron

In [ ]:
# @title
# example using a neuron

# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(1.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(0.0,  label='w2')
# bias of the neuron
b = Value(8.0, label='b')
# x1w1 + x2w2 + b
x1w1 = x1*w1; x1w1.label = 'x1w1'
x2w2 = x2*w2; x2w2.label = 'x2w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1w1 + x2w2'
n = x1w1x2w2 + b; n.label='n'

o = n.tanh(); o.label ='o'
o.backward()
draw_dot(o)


In [ ]:
# @title
# example using a neuron

# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(1.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(0.0,  label='w2')
# bias of the neuron
b = Value(8.0, label='b')
# x1w1 + x2w2 + b
x1w1 = x1*w1; x1w1.label = 'x1w1'
x2w2 = x2*w2; x2w2.label = 'x2w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1w1 + x2w2'
n = x1w1x2w2 + b; n.label='n'
# ---------------
e = (2*n).exp()
o = (e-1)/(e+1)
o.label = 'o'
o.backward()
draw_dot(o)
#--------------

In [ ]:
# @title back-propagation
# Automated back-propagation
o.grad = 1.0
o._backward()
n._backward()
b._backward()
x1w1x2w2._backward()
x1w1._backward()
x2w2._backward()
# Now we need to automate calling ._backward

In [ ]:
# @title TOPOLOGICAL SORT
# we must only call ._backward when the entire forward pass is done, and reorder from output to input
# i.e all the nodes & edges of the graph have been assigned values
# this can be done by 'TOPOLOGICAL SORT'
topo = []
visited = set()
def build_topo(v):
  if v not in visited:
    visited.add(v)

    for child in v._prev:

      build_topo(child)

    topo.append(v)
build_topo(o)
topo

In [ ]:
# @title
a = Value(3.0, label='a')
b = a + a; b.label = 'b'
b.backward()
draw_dot(b)

In [ ]:
# @title
a = Value(-2.0, label='a')
b = Value(3.0, label='b')
d = a*b; d.label='d'
e = a+b; e.label='e'
f = d*e; f.label='f'
f.backward()
draw_dot(f)

In [ ]:
# @title manual back-propagation
# manual back-propagation
o.grad = 1.0
# o = tanh(n)
n.grad = 1 - o.data **2 # do/dn = 1 - o**2
x1w1x2w2.grad = n.grad * 1.0 # do/d(x1w1x2w2) = do/dn * dn/d(x1w1x2w2)
b.grad = n.grad * 1.0 # do/db = do/dn * dn/db
x1w1.grad = x1w1x2w2.grad * 1.0 # do/dx1w1 = do/dn * dn/d(x1w1x2w2) * d(x1w1x2w2)/dx1w1
x2w2.grad = x1w1x2w2.grad * 1.0 # do/dx2w2 = do/dn * dn/d(x1w1x2w2) * d(x1w1x2w2)/dx2w2

x1.grad = x1w1.grad * w1.data
w1.grad = x1w1.grad * x1.data # do/dw1 = do/dn * dn/d(x1w1x2w2) * d(x1w1x2w2)/dx1w1 * dx1w1/dw1
x2.grad = x1w1.grad * w2.data
w2.grad = x2w2.grad * x2.data # do/dw2 = do/dn * dn/d(x1w1x2w2) * d(x1w1x2w2)/dx2w2 * dx2w2/dw2

# ***USING PYTORCH***  

In [ ]:
import torch

In [ ]:
# @title
x1 = torch.Tensor([2.0]).double()                     ; x1.requires_grad = True
x2 = torch.Tensor([0.0]).double()                     ; x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double()                    ; w1.requires_grad = True
w2 = torch.Tensor([1.0]).double()                     ; w2.requires_grad = True
b = torch.Tensor([6.88135735870195432]).double()      ; b.requires_grad  = True
n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

print(o.data.item())
o.backward()

print("--------------")
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('w1', w1.grad.item())


In [ ]:
torch.Tensor([2.0]).dtype

torch.float32

In [ ]:
torch.Tensor([2.0]).double().dtype

torch.float64

In [ ]:
o

tensor([0.7071], dtype=torch.float64, grad_fn=<TanhBackward0>)

In [ ]:
o.item()

0.7070985840783353

# **Implementing MLP with micrograd**

In [ ]:
np.random.uniform(-1,1)

0.2418644333566824

In [ ]:
class Neuron:

  def __init__(self, nin):
    self.w = [Value(np.random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(np.random.uniform(-1,1))

  def __call__(self, x):
    # x = [2.0, 3.0]
    # n = Neuron(2) # initialize a neuron object with 2 inputs
    # n(x) here python will use __call__ function
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out

  def parameters(self):
    return self.w + [self.b]

class Layer:

  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]

  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs

  def parameters(self):
    # params = []
    # for neuron in self.neurons:
    #   ps = neuron.parameters()
    #   params.extend(ps)
    # return params
    return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

  def __call__(self, x):
    for layer in self.layers:
      x = layer(x) # here x updates so we are not inputting same x for every layer
    return x

  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]



In [ ]:
#x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
#n(x) # here python will use __call__ function

In [ ]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired target


In [ ]:
n.parameters() # weights and biases

[Value(data=-0.10632380513402562),
 Value(data=-0.9272302798541971),
 Value(data=0.6330461089538579),
 Value(data=-0.7888907226951398),
 Value(data=0.04692350932016831),
 Value(data=-0.1106001008947679),
 Value(data=-0.36618031901594517),
 Value(data=-0.33749055002628037),
 Value(data=-0.9954642550530362),
 Value(data=0.7377405763133),
 Value(data=0.4236308346653055),
 Value(data=0.7735155768306972),
 Value(data=-0.7766641102502208),
 Value(data=0.1314689464855885),
 Value(data=-0.8171003322597794),
 Value(data=0.5546771944342328),
 Value(data=0.7728321784270007),
 Value(data=0.1506235817543522),
 Value(data=-0.837837418037702),
 Value(data=-0.048455832216313555),
 Value(data=0.9979793701323036),
 Value(data=-0.795991784796197),
 Value(data=-0.3924557138041571),
 Value(data=0.9841825496110828),
 Value(data=-0.007332303560053655),
 Value(data=-0.42058075822778207),
 Value(data=-0.5070202162935493),
 Value(data=0.5910236478410194),
 Value(data=-0.45449653111508925),
 Value(data=-0.101417

In [ ]:
for k in range(20):
  # forward pass
  ypred = [n(x) for x in xs]
  loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))

  # backward
  for p in n.parameters():
    p.grad = 0.0  # resets the gradients to 0 after each weights, bias update
                  # otherwise the previous gradients would accumulate
                  # new weights and biases will give new gradients when backpropping, not accumulate with old ones
  loss.backward()

  # update
  for p in n.parameters():
    p.data += -0.05 * p.grad


  print(k, loss.data)

0 0.01218490014859194
1 0.011942034351282237
2 0.011708421875058007
3 0.011483551377520053
4 0.011266948139715526
5 0.011058170868982111
6 0.010856808829054734
7 0.010662479259188907
8 0.010474825049061581
9 0.010293512640490337
10 0.010118230130685852
11 0.009948685554910602
12 0.00978460532914135
13 0.00962573283568365
14 0.009471827136727312
15 0.00932266180259785
16 0.009178023842999075
17 0.009037712730881785
18 0.008901539509745541
19 0.00876932597620389


In [ ]:
ypred

[Value(data=0.9549348612420521),
 Value(data=-0.9911360735543078),
 Value(data=-0.9389633598576318),
 Value(data=0.9458297257434745)]

In [ ]:
draw_dot(loss)